# 세이지메이커 LMI와 Rolling Batch를 활용해 세이지메이커에서 높은 성능으로 Llama2 70B 모델 배포

이 노트북에서는 DeepSpeed를 활용하여 세이지메이커에서 FP16 정밀도로 Llama2 대규모 언어 모델을 호스팅하는 방법을 탐구합니다. 이 예시에서는 LMI 컨테이너에 포함된 DJLServing을 모델 서빙 솔루션으로 활용합니다. DJLServing은 Deep Java Library (DJL)를 기반으로 하는 고성능의 범용 모델 서빙 솔루션으로, 프로그래밍 언어에 구애받지 않습니다. DJL과 DJLServing에 대한 더 자세한 내용은 최근 블로그 포스트에서 확인할 수 있습니다(https://aws.amazon.com/blogs/machine-learning/deploy-bloom-176b-and-opt-30b-on-amazon-sagemaker-with-large-model-inference-deep-learning-containers-and-deepspeed/).

모델 병렬성은 일반적으로 단일 GPU로는 너무 큰 모델을 배포하는 데 도움을 줄 수 있습니다. 모델 병렬성을 활용하면 모델을 여러 GPU에 분할하고 분산시킵니다. 각 GPU는 모델의 서로 다른 부분을 보유하며, 이는 수십억 개의 매개변수를 가진 가장 큰 딥러닝 모델의 메모리 사용량 문제를 해결합니다.

세이지메이커는 이제 DeepSpeed 컨테이너를 롤아웃했으며, 사용자가 관리형 서빙 기능을 활용하고, 비즈니스에 차별화되지 않은 무거운 작업을 제공하는 데 도움을 줍니다.

이 노트북에서는 https://huggingface.co/TheBloke/Llama-2-70b-fp16 모델을 ml.g5.48xlarge 인스턴스에서 배포합니다.

# 라이센스 동의
 - 모델 사용 전에 라이센스 정보를 [여기](https://huggingface.co/meta-llama)에서 확인할 수 있습니다.
 - 이 노트북은 샘플 노트북이며 운영 용도가 아닙니다. 라이센스는 [여기](https://github.com/aws/mit-0)에서 확인할 수 있습니다.

In [ ]:
!pip install sagemaker boto3 --upgrade

In [ ]:
import sagemaker
import jinja2
from sagemaker import image_uris
import boto3
import os
import time
import json
from pathlib import Path

In [ ]:
role = sagemaker.get_execution_role()  # 엔드포인트 실행 역할
sess = sagemaker.session.Session()  # 다양한 AWS API와 상호작용하기 위한 세이지메이커 세션
bucket = sess.default_bucket()  # 아티팩트를 저장할 버킷

In [ ]:
model_bucket = sess.default_bucket()  # 아티팩트를 저장할 버킷
s3_code_prefix = "hf-large-model-djl/meta-llama/Llama-2-70b-fp16/code"  # 코드 아티팩트가 들어갈 버킷 내 폴더

s3_model_prefix = "hf-large-model-djl/meta-llama/Llama-2-70b-fp16/model"  # 코드 아티팩트가 들어갈 버킷 내 폴더
region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sagemaker_client = boto3.client("sagemaker")
sagemaker_runtime_client = boto3.client("sagemaker-runtime")

jinja_env = jinja2.Environment()

### 모델이 위치한 S3 URL을 포함하는 변수를 정의합니다.

In [ ]:
# 모델이 위치한 S3 URL을 포함하는 변수를 정의합니다. 데모 목적으로, S3 버킷에서 Llama-2-70b-fp16 모델 아티팩트를 활용합니다.
pretrained_model_location = f"s3://sagemaker-example-files-prod-{region}/models/llama-2/fp16/70B/"

## 세이지메이커 호환 모델 아티팩트 생성, 모델을 S3에 업로드하고 자체 추론 스크립트 가져오기

세이지메이커 대규모 모델 추론 컨테이너는 자체 추론 코드를 제공하지 않고도 모델을 호스팅하는 데 활용할 수 있습니다. 이는 입력 데이터의 사용자 정의 전처리나 모델의 예측 결과에 대한 후처리가 필요 없는 경우에 매우 유용합니다.

세이지메이커는 모델 아티팩트가 Tarball 형식이어야 합니다. 이 예시에서는 다음 파일을 제공합니다.

```
code
├──── 
│   └── serving.properties
```

    serving.properties는 모델 서버 구성 파일입니다.


#### serving.properties 생성
이 파일은 DJL Serving에 활용할 모델 병렬화 및 추론 최적화 라이브러리를 지정하는 구성 파일입니다. 필요에 따라 적절한 구성을 설정할 수 있습니다.

이 구성 파일의 설정 목록은 다음과 같습니다.

    engine: DJL이 활용할 엔진을 지정합니다. 이 경우, MPI로 설정했습니다.
    option.model_id: 사전 학습된 모델의 ID를 지정합니다. 이는 huggingface.co의 모델 저장소에 호스팅된 모델 ID이거나 모델 아티팩트의 S3 경로일 수 있습니다. 
    option.tensor_parallel_degree: 모델을 분할하는 데 필요한 GPU 장치의 수를 설정합니다. 이 매개변수는 DJL serving이 실행될 때 시작되는 모델당 워커 수를 제어합니다. 예를 들어, 4개의 GPU 머신에서 4개의 파티션을 생성하는 경우, 요청을 처리하기 위해 모델당 1개의 워커가 시작됩니다.

구성 옵션 및 포괄적인 목록에 대한 자세한 내용은 [문서](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-large-model-configuration.html)에서 확인할 수 있습니다.

In [ ]:
!rm -rf code_llama2_70b_fp16
!mkdir -p code_llama2_70b_fp16

In [ ]:
%%writefile code_llama2_70b_fp16/serving.properties
engine = MPI
option.tensor_parallel_degree = 8
option.rolling_batch = auto
option.max_rolling_batch_size = 4
option.model_loading_timeout = 3600
option.model_id = {{model_id}}
option.paged_attention = true
option.trust_remote_code = true
option.dtype = fp16

In [ ]:
# `serving.properties` 파일에 적절한 모델 위치를 설정합니다.
template = jinja_env.from_string(Path("code_llama2_70b_fp16/serving.properties").open().read())
Path("code_llama2_70b_fp16/serving.properties").open("w").write(
    template.render(model_id=pretrained_model_location)
)
!pygmentize code_llama2_70b_fp16/serving.properties | cat -n

**DJL 컨테이너의 이미지 URI를 활용합니다.**

In [ ]:
inference_image_uri = image_uris.retrieve(
    framework="djl-deepspeed", region=region, version="0.23.0"
)
print(f"Image going to be used is ---- > {inference_image_uri}")

**Tarball을 생성한 후 S3 위치에 업로드합니다.**

In [ ]:
!rm model.tar.gz
!tar czvf model.tar.gz code_llama2_70b_fp16

In [ ]:
s3_code_artifact = sess.upload_data("model.tar.gz", bucket, s3_code_prefix)

### 엔드포인트를 생성하는 단계는 다음과 같습니다.

1. 이미지 컨테이너와 이전에 업로드한 모델 Tarball을 활용하여 모델을 생성합니다.
2. 다음 주요 매개변수를 사용하여 엔드포인트 구성을 생성합니다.

    a) 인스턴스 유형은 `ml.g5.48xlarge`입니다.
    
    b) `ContainerStartupHealthCheckTimeoutInSeconds`는 3600으로 설정하여 모델이 준비된 후 헬스 체크가 시작되도록 합니다.
3. 생성한 엔드포인트 구성으로 엔드포인트를 생성합니다.

#### 모델 생성
DJL 컨테이너의 이미지 URI와 Tarball이 업로드된 S3 위치를 활용합니다.

컨테이너는 모델을 인스턴스의 `/tmp` 공간에 다운로드합니다. 세이지메이커는 `/tmp`를 아마존 EBS(Amazon Elastic Block Store) 볼륨에 매핑하며, 이 볼륨은 엔드포인트 생성 매개변수 `VolumeSizeInGB`를 지정할 때 마운트됩니다. 컨테이너는 매우 빠른 다운로드 속도를 제공하는 `s5cmd`(https://github.com/peak/s5cmd)를 활용하여 대규모 모델 다운로드에 유용합니다.

p4dn과 같이 볼륨 인스턴스가 사전 구축된 경우, 컨테이너의 `/tmp`를 계속 활용할 수 있습니다. 이 마운트의 크기는 모델을 저장하기에 충분히 큽니다.

In [ ]:
from sagemaker.utils import name_from_base

model_name = name_from_base(f"Llama-2-70b-fp16-mpi")
print(model_name)

create_model_response = sagemaker_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        "Image": inference_image_uri,
        "ModelDataUrl": s3_code_artifact,
        "Environment": {"MODEL_LOADING_TIMEOUT": "3600"},
    },
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

In [ ]:
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"

endpoint_config_response = sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g5.48xlarge",
            "InitialInstanceCount": 1,
            "ModelDataDownloadTimeoutInSeconds": 3600,
            "ContainerStartupHealthCheckTimeoutInSeconds": 3600,
        },
    ],
)
endpoint_config_response

In [ ]:
create_endpoint_response = sagemaker_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">SageMaker REST Endpoint</a></b>'.format(
            region, endpoint_name
        )
    )
)

### 이 단계는 약 20분 이상 소요될 수 있으니, 인내심을 가지고 기다려 주시기 바랍니다.

In [ ]:
import time

resp = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

#### 엔드포인트가 생성될 때까지 기다리는 동안, 다음 내용을 더 읽어볼 수 있습니다.
- [대규모 모델 추론을 위한 딥러닝 컨테이너](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-large-model-dlc.html)

#### Boto3를 활용하여 엔드포인트 호출

이 모델은 생성 모델이므로, 텍스트를 프롬프트로 전달하면 모델이 문장을 완성하고 결과를 반환합니다.

모델에 프롬프트를 입력으로 전달할 수 있습니다. 이는 입력을 프롬프트로 설정하여 수행됩니다. 모델은 각 프롬프트에 대해 결과를 반환합니다. 텍스트 생성은 적절한 매개변수를 활용하여 구성할 수 있습니다. 이러한 매개변수는 kwargs 사전으로 엔드포인트에 전달해야 합니다. 자세한 내용은 [문서](https://huggingface.co/docs/transformers/main/en/main_classes/text_generation#transformers.GenerationConfig)에서 확인할 수 있습니다.

아래 코드 샘플은 텍스트 프롬프트를 활용하여 엔드포인트를 호출하고 몇 가지 매개변수를 설정하는 방법을 보여줍니다.

In [ ]:
sagemaker_runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(
        {
            "inputs": "The diamondback terrapin was the first reptile to do what?",
            "parameters": {
                "do_sample": True,
                "max_new_tokens": 100,
                "min_new_tokens": 100,
                "temperature": 0.3,
                "watermark": True,
            },
        }
    ),
    ContentType="application/json",
)["Body"].read().decode("utf8")

# 세이지메이커 엔드포인트 오토스케일링

In [ ]:
autoscale = boto3.Session().client(service_name="application-autoscaling")

In [ ]:
autoscale.register_scalable_target(
    ServiceNamespace="sagemaker",
    ResourceId="endpoint/" + endpoint_name + "/variant/variant1",
    ScalableDimension="sagemaker:variant:DesiredInstanceCount",
    MinCapacity=1,
    MaxCapacity=2,
    RoleARN=role,
    SuspendedState={
        "DynamicScalingInSuspended": False,
        "DynamicScalingOutSuspended": False,
        "ScheduledScalingSuspended": False,
    },
)

In [ ]:
# 대상이 사용 가능한지 확인합니다.
autoscale.describe_scalable_targets(
    ServiceNamespace="sagemaker",
    MaxResults=100,
)

In [ ]:
autoscale.put_scaling_policy(
    PolicyName="autoscale-policy-gpu-400-llama2-70b",
    ServiceNamespace="sagemaker",
    ResourceId="endpoint/" + endpoint_name + "/variant/variant1",
    ScalableDimension="sagemaker:variant:DesiredInstanceCount",
    PolicyType="TargetTrackingScaling",
    TargetTrackingScalingPolicyConfiguration={
        "TargetValue": 400, # 총 GPU 활용도를 400%/800% (8 GPUs)로 설정합니다.
        "CustomizedMetricSpecification":
        {
            "MetricName": "GPUUtilization",
            "Namespace": "/aws/sagemaker/Endpoints",
            "Dimensions": [
                {"Name": "EndpointName", "Value": endpoint_name},
                {"Name": "VariantName", "Value": "variant1"}
            ],
            "Statistic": "Average",
            "Unit": "Percent"
        },
        "ScaleOutCooldown": 60,
        "ScaleInCooldown": 300,
    }
)

# 오토스케일링 실행

In [ ]:
for i in range(0, 100):
    res = sagemaker_runtime_client.invoke_endpoint(
        EndpointName=endpoint_name,
        Body=json.dumps(
            {
                "inputs": "The diamondback terrapin was the first reptile to do what?",
                "parameters": {
                    "do_sample": True,
                    "max_new_tokens": 100,
                    "min_new_tokens": 100,
                    "temperature": 0.3,
                    "watermark": True,
                },
            }
        ),
        ContentType="application/json",
    )
    print(f'{i}: {res["Body"].read().decode("utf8")}')

In [ ]:
autoscale.describe_scaling_activities(
    ServiceNamespace="sagemaker",
    ResourceId="endpoint/" + endpoint_name + "/variant/variant1",
    ScalableDimension="sagemaker:variant:DesiredInstanceCount",
    MaxResults=100
)

## 정리

In [ ]:
# # - 엔드 포인트 제거
# sagemaker_client.delete_endpoint(EndpointName=endpoint_name)

In [1]:
# # - 엔드포인트가 실패할 경우, 모델을 삭제하는 것도 고려해야 합니다.
# sagemaker_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
# sagemaker_client.delete_model(ModelName=model_name)